## W205 Project 2

The business question is to prepare the infrastructure to land the data in the correct form in order to query it. Everything I did in the command line is shown below and labeled (I completed this more for my own sake of keepings things straight in the command line). I also have some questions to explore from the queries that I will answer at the end of the report. 

#### Questions: 
- What are the statistics of the exams?
- How many certifications were false and true?
- How many exams were there and how many different exams? 

##### Notes:
- I did change the data file name to "data" so you will see the file "data.json" loaded in, this is the correct one from the project 2 file.
- For the command line history file, the project commands start at line 210 - before that was a lot of practice and does not go in a correct order. 

In [2]:
#download data into w205 file
curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp

#Check and if necessary, modify your docker-compose.yml file
mkdir project_2
#save correct YAML file into this file

#move into correct file
cd ~/w205/project2

#TO BEGIN
docker-compose up -d
docker-compose ps
docker ps -a

#Create a symbolic link in the Spark container to the /205 mount point
#First exec a bash shell into the spark container:
docker-compose exec spark bash
#Create a symbolic link from the spark directory to /w205 :
ln -s /w205 w205
#then exit the container
exit

####dont have to do next two lines - I did not I did it all in the command line
#Run an enhanced version of the pyspark command line to target Jupyter Notebook
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

#URL if planning on running pyspark in a jupyter notebook - can also just do it from the command line
#34.123.4.189:8888 - google clous external IP address 
http://34.123.4.189:8888/?token=7520af10bd9961be5bee06e3a7eb3a135bad463f961e9b08

#Let's check out hdfs before we write anything to it
docker-compose exec cloudera hadoop fs -ls /tmp/

#create topic - data
docker-compose exec kafka kafka-topics --create --topic data --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

#use kafkacat to product messages to data topic
docker-compose exec mids \
  bash -c "cat /w205/data.json \
    | jq '.[]' -c \
    | kafkacat -P -b kafka:29092 -t data"

#spring up a pyspark process using the spark container
docker-compose exec spark pyspark

#at the pyspark prompt, read from kafka
raw_data = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","data") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 

#cache this to cut back on warnings later
raw_data.cache()

#lets explore
raw_data.printSchema()

#working on unrolling json
data = raw_data.select(raw_data.value.cast('string'))

data.select('value').take(1)

data.select('value').take(1)[0].value

import json

from pyspark.sql import Row

extracted_data = data.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

extracted_data.registerTempTable('data')

spark.sql("select keen_id from data limit 20").show()

spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from data limit 10").show()

spark.sql("select exam_name, sequences.questions[0].user_incomplete from data limit 10").show()

spark.sql("select count(certification) from data where certification == false").show()

spark.sql("select count(certification) from data where certification == true").show()

spark.sql("select count(distinct exam_name) from data").show()

spark.sql("select sequences.abc123 from data limit 10").show()

def my_lambda_sequences_id(x):
   raw_dict = json.loads(x.value)
   my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
   return Row(**my_dict)

my_sequences = data.rdd.map(my_lambda_sequences_id).toDF()

my_sequences.registerTempTable('sequences')

spark.sql("select sequences_id from sequences limit 10").show()

spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from data a join sequences s on a.keen_id = s.keen_id limit 10").show()

              
def my_lambda_questions(x):
   raw_dict = json.loads(x.value)
   my_list = []
   my_count = 0
   for l in raw_dict["sequences"]["questions"]:
       my_count += 1
       my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
       my_list.append(Row(**my_dict))
   return my_list


my_questions = data.rdd.flatMap(my_lambda_questions).toDF()

my_questions.registerTempTable('questions')

spark.sql("select id, my_count from questions limit 10").show()

spark.sql("select q.keen_id, a.keen_timestamp, q.id from data a join questions q on a.keen_id = q.keen_id limit 10").show()

def my_lambda_correct_total(x):
   raw_dict = json.loads(x.value)
   my_list = []
   if "sequences" in raw_dict:
       if "counts" in raw_dict["sequences"]:
           if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
               my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"],
                          "total": raw_dict["sequences"]["counts"]["total"]}
               my_list.append(Row(**my_dict))
   return my_list

my_correct_total = data.rdd.flatMap(my_lambda_correct_total).toDF()

my_correct_total.registerTempTable('ct')

spark.sql("select * from ct limit 10").show()

spark.sql("select correct / total as score from ct limit 10").show()

spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()

spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

#exit pyspark when completed
#save pypark command history
docker-compose exec spark cat /root/.python_history

#check to make sure stuff is there again after writing to hdfs
docker-compose exec cloudera hadoop fs -ls /tmp/
docker-compose exec cloudera hadoop fs -ls /tmp/data
docker-compose exec cloudera hadoop fs -ls /tmp/my_questions
docker-compose exec cloudera hadoop fs -ls /tmp/my_correct_total

#Then close it down
docker-compose down
docker-compose ps

#When absolutely finished:
history >mpugel-history.txt

#resources for unrolling Json
#https://kontext.tech/column/spark/284/pyspark-convert-json-string-column-to-array-of-object-structtype-in-data-frame
#https://www.tutorialspoint.com/spark_sql/spark_sql_json_datasets.htm



SyntaxError: invalid syntax (<ipython-input-2-d3494f6eed35>, line 2)

#### Answering Questions Based on Queries:
- What are the statistics of the exams?
        - The average score was 62.65%
        - The standard deviations was 0.311
- How many certifications were false and true?
        - False: 3148
        - True: 0
- How many exams were there and how many different exams? 
         - 3280 total exams
         - 103 different exams

#### Conclusion
Overall, there is a lot given in this file, there is all the data about the user taking the exam, and many ids to keep track of users, questions, and exams. There is also a lot of information on questions in the exams and the users results. There were some assumptions I made about the data that I'm not sure where completely correct. I assumed the "certifications" section would indicate if the user "passed" the exam and was certified, however based on my query results I do not believe this to be the case anymore. The biggest issue with the data I found is I had no context to it, and therefore I had to assume what certain entries meant. Also, the structure of the json file was difficult to deal with and in order to get to different layers of the data I need to use functions to unroll the json file. 

In [ ]:
#docker compose file
#https://docs.divio.com/en/latest/reference/docker-docker-compose/
---
version: '2'
services:
  zookeeper:
    image: confluentinc/cp-zookeeper:latest
    environment:
      ZOOKEEPER_CLIENT_PORT: 32181
      ZOOKEEPER_TICK_TIME: 2000
    expose:
      - "2181"
      - "2888"
      - "32181"
      - "3888"

  kafka:
    image: confluentinc/cp-kafka:latest
    depends_on:
      - zookeeper
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:32181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:29092
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
    expose:
      - "9092"
      - "29092"

  cloudera:
    image: midsw205/cdh-minimal:latest
    expose:
      - "8020" # nn
      - "50070" # nn http

  spark:
    image: midsw205/spark-python:0.0.5
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205
    command: bash
    depends_on:
      - cloudera
    environment:
      HADOOP_NAMENODE: cloudera
    expose:
      - "8888"
    ports:
      - "8888:8888"

  mids:
    image: midsw205/base:latest
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205


In [ ]:
### history from pyspark - copied from command line after running docker-compose exec spark cat /root/.python_history
raw_data = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","data") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 
raw_data.cache()
raw_data.printSchema()
data = raw_data.select(raw_data.value.cast('string'))
data.select('value').take(1)
data.select('value').take(1)[0].value
import json
from pyspark.sql import Row
extracted_data = data.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()
extracted_data.registerTempTable('data')
spark.sql("select keen_id from data limit 20").show()
spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from data limit 10").show()
spark.sql("select exam_name, sequences.questions[0].user_incomplete from data limit 10").show()
spark.sql("select sequences.abc123 from data limit 10").show()
spark.sql("select count(certification) from data where certification == False").show()
spark.sql("select count(certification) from data where certification == True").show()
spark.sql("select count(certification) from data where certification == false").show()
spark.sql("select count(certification) from data where certification == true").show()
spark.sql("select count(unique exam_name) from data").show()
spark.sql("select count(exam_name) from data").show()
spark.sql("select count(unique(exam_name)) from data").show()
spark.sql("select count(distinct exam_name) from data").show()
def my_lambda_questions(x):
   raw_dict = json.loads(x.value)
   my_list = []
   my_count = 0
   for l in raw_dict["sequences"]["questions"]:
       my_count += 1
       my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
       my_list.append(Row(**my_dict))
   return my_list
my_questions = data.rdd.flatMap(my_lambda_questions).toDF()
my_questions.registerTempTable('questions')
spark.sql("select id, my_count from questions limit 10").show()
spark.sql("select q.keen_id, a.keen_timestamp, q.id from data a join questions q on a.keen_id = q.keen_id limit 10").show()
def my_lambda_correct_total(x):
   raw_dict = json.loads(x.value)
   my_list = []
   if "sequences" in raw_dict:
       if "counts" in raw_dict["sequences"]:
           if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
               my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"],
                          "total": raw_dict["sequences"]["counts"]["total"]}
               my_list.append(Row(**my_dict))
   return my_list
my_correct_total = data.rdd.flatMap(my_lambda_correct_total).toDF()
my_correct_total.registerTempTable('ct')
spark.sql("select * from ct limit 10").show()
spark.sql("select correct / total as score from ct limit 10").show()
spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()
spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()
exit
exit()